In [3]:
import pandas as pd
import csv
import pymongo
import json
%load_ext blackcellmagic
from sqlalchemy import create_engine
from db_config import engine, DATABASE

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [ ]:
us_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
us_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
global_cases = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
global_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
global_recovered = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

In [ ]:
us_confirmed_df = pd.read_csv(us_confirmed)
us_deaths_df = pd.read_csv(us_deaths)
global_cases_df = pd.read_csv(global_cases)
global_deaths_df = pd.read_csv(global_deaths)
global_recovered_df = pd.read_csv(global_recovered)

In [16]:
def insert_records(table_name, df):
    engine.execute(f"DROP TABLE IF EXISTS {table_name}")
    df.to_sql(name=table_name, con=engine)

## US DATA

In [20]:
states_data = pd.DataFrame(
    us_deaths_df[
        ["UID", "iso2", "Admin2", "Province_State", "Population", "Lat", "Long_"]
    ]
).rename(columns={"iso2": "Country_Abbrv", "Admin2": "County", "Long_": "Long"})
states_data

,UID,Country_Abbrv,County,Province_State,Population,Lat,Long
0,16,AS,NaN,American Samoa,55641,-14.2710,-170.1320
1,316,GU,NaN,Guam,164229,13.4443,144.7937
2,580,MP,NaN,Northern Mariana Islands,55144,15.0979,145.6739
3,630,PR,NaN,Puerto Rico,2933408,18.2208,-66.5901
4,850,VI,NaN,Virgin Islands,107268,18.3358,-64.8963
...,...,...,...,...,...,...,...
3248,84090053,US,Unassigned,Washington,0,0.0000,0.0000
3249,84090054,US,Unassigned,West Virginia,0,0.0000,0.0000
3250,84090055,US,Unassigned,Wisconsin,0,0.0000,0.0000
3251,84090056,US,Unassigned,Wyoming,0,0.0000,0.0000


In [ ]:
# insert_records("state_data", states_data)

In [ ]:
us_cases_clean = pd.melt(
    us_confirmed_df.drop(
        columns=[
            "iso2",
            "iso3",
            "code3",
            "FIPS",
            "Admin2",
            "Province_State",
            "Country_Region",
            "Lat",
            "Long_",
            "Combined_Key",
        ]
    ),
    id_vars="UID",
    var_name="Date",
).rename(columns={"value": "Confirmed_Cases"})

us_cases_clean

In [ ]:
us_deaths_clean = pd.melt(
    us_deaths_df.drop(
        columns=[
            "iso2",
            "iso3",
            "code3",
            "FIPS",
            "Admin2",
            "Province_State",
            "Country_Region",
            "Lat",
            "Long_",
            "Combined_Key",
            "Population",
        ]
    ),
    id_vars="UID",
    var_name="Date",
).rename(columns={"value": "Deaths"})

us_deaths_clean

In [ ]:
us_covid_data = pd.merge(
    us_cases_clean, us_deaths_clean, how="left", on=["UID", "Date"]
).fillna(0)
us_covid_data

## GLOBAL DATA

In [ ]:
global_cases_clean = pd.melt(
    global_cases_df,
    id_vars=["Province/State", "Country/Region", "Lat", "Long"],
    var_name="Date",
).rename(columns={"value": "Confirmed_Cases"})

global_cases_clean

In [ ]:
global_deaths_clean = pd.melt(
    global_deaths_df,
    id_vars=["Province/State", "Country/Region", "Lat", "Long"],
    var_name="Date",
).rename(columns={"value": "Deaths"})

global_deaths_clean

In [ ]:
global_recovered_clean = pd.melt(
    global_recovered_df,
    id_vars=["Province/State", "Country/Region", "Lat", "Long"],
    var_name="Date",
).rename(columns={"value": "Recovered"})

global_recovered_clean

In [21]:
global_covid_data = pd.merge(
    pd.merge(
        global_cases_clean,
        global_deaths_clean,
        how="left",
        on=["Province/State", "Country/Region", "Lat", "Long", "Date"],
    ),
    global_recovered_clean,
    how="left",
    on=["Province/State", "Country/Region", "Lat", "Long", "Date"],
).rename(
    columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}
).fillna(0)
global_covid_data

,Province_State,Country_Region,Lat,Long,Date,Confirmed_Cases,Deaths,Recovered
0,0,Afghanistan,33.00000,65.000000,1/22/20,0,0,0.0
1,0,Albania,41.15330,20.168300,1/22/20,0,0,0.0
2,0,Algeria,28.03390,1.659600,1/22/20,0,0,0.0
3,0,Andorra,42.50630,1.521800,1/22/20,0,0,0.0
4,0,Angola,-11.20270,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
20246,Falkland Islands (Malvinas),United Kingdom,-51.79630,-59.523600,4/7/20,2,0,0.0
20247,Saint Pierre and Miquelon,France,46.88520,-56.315900,4/7/20,1,0,0.0
20248,0,South Sudan,6.87700,31.307000,4/7/20,2,0,0.0
20249,0,Western Sahara,24.21550,-12.885800,4/7/20,4,0,0.0
